# Création d'un dictionnaire qui contient un tableau pour chaque ligne de métro

## Chaque tableau affiche les stations de la ligne, leur fréquentaiton annuelle, leur fréquentation hebdomadaire moyenne sur la période 24/07/2022-11/08/2022 (période des JO mais en 2022)

In [2]:
import pandas as pd
import numpy as np
import re
import difflib

Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

In [3]:
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

/tmp/ipykernel_6154/1420814720.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


Importation ableau avec station, ligne, correspondances... (Données disponibles sur le site https://data.ratp.fr)

In [4]:
%pip install openpyxl
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_fréquentation = tableau_fréquentation.drop_duplicates(subset='Station')


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Nettoyage du tableau

In [5]:
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

print(tableau_fréquentation)

     Rang                 Station    Trafic Correspondance_1 Correspondance_2  \
0       7              REPUBLIQUE  11079708                3                5   
1       9              LA DEFENSE   9256802                1              nan   
2      13          HOTEL DE VILLE   7251729                1               11   
3      30  VILLEJUIF-LOUIS ARAGON   5218070                7              nan   
4      40            PALAIS-ROYAL   4822599                1                7   
..    ...                     ...       ...              ...              ...   
357   286                  PICPUS    931602                6              nan   
358   287                  TEMPLE    891858                3              nan   
359   289        QUAI DE LA RAPEE    798728                5              nan   
360   299                  DANUBE    400157             7bis              nan   
361   303       PRE-SAINT-GERVAIS    282626             7bis              nan   

    Correspondance_3 Corres

In [6]:
# Ajouter la colonne 'nb_correspondance'
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].apply(lambda row: sum(pd.notna(value) and value != 'nan' for value in row), axis=1)
print(tableau_fréquentation)

     Rang                 Station    Trafic Correspondance_1 Correspondance_2  \
0       7              REPUBLIQUE  11079708                3                5   
1       9              LA DEFENSE   9256802                1              nan   
2      13          HOTEL DE VILLE   7251729                1               11   
3      30  VILLEJUIF-LOUIS ARAGON   5218070                7              nan   
4      40            PALAIS-ROYAL   4822599                1                7   
..    ...                     ...       ...              ...              ...   
357   286                  PICPUS    931602                6              nan   
358   287                  TEMPLE    891858                3              nan   
359   289        QUAI DE LA RAPEE    798728                5              nan   
360   299                  DANUBE    400157             7bis              nan   
361   303       PRE-SAINT-GERVAIS    282626             7bis              nan   

    Correspondance_3 Corres

In [7]:
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
print(tableau_fréquentation)

     Rang                 Station     Trafic Correspondance_1  \
0       7              REPUBLIQUE  2215941.6                3   
1       9              LA DEFENSE  9256802.0                1   
2      13          HOTEL DE VILLE  3625864.5                1   
3      30  VILLEJUIF-LOUIS ARAGON  5218070.0                7   
4      40            PALAIS-ROYAL  2411299.5                1   
..    ...                     ...        ...              ...   
357   286                  PICPUS   931602.0                6   
358   287                  TEMPLE   891858.0                3   
359   289        QUAI DE LA RAPEE   798728.0                5   
360   299                  DANUBE   400157.0             7bis   
361   303       PRE-SAINT-GERVAIS   282626.0             7bis   

    Correspondance_2 Correspondance_3 Correspondance_4 Correspondance_5  \
0                  5                8              9.0             11.0   
1                nan              nan              nan              n

Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation hebdomadaire à chaque station

In [8]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.315789
1,ablon,1101.105263
2,acheres grand,51.388889
3,acheres ville,3012.052632
4,aeroport cdg1,5050.947368
...,...,...
733,vosves,14.894737
734,vulaines sur se,42.894737
735,wagram,3268.157895
736,yerres,2743.000000


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne 

In [9]:
from IPython.display import display

numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in str(x.values), axis=1)]
    
    # Supprimer les doublons dans la colonne 'Station'
    filtered_rows = filtered_rows.drop_duplicates(subset='Station')
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')
    tableaux[tableau_nom] = tableau.drop_duplicates(subset='Station')  # Ajoutez cette ligne pour supprimer les doublons de la colonne 'Station'


In [10]:

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [11]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [12]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [13]:

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les doublons de stations
    current_table = current_table.drop_duplicates(subset='Station')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table


In [14]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table

In [15]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table


In [16]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic',
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne',
    'fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")
print(tableau.columns)
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")

Colonnes disponibles :
Index(['Rang', 'Station', 'Trafic', 'Correspondance_1', 'Correspondance_2',
       'Correspondance_3', 'Correspondance_4', 'Correspondance_5',
       'nb_correspondance', 'fréquentation_totale_ligne',
       'Part du trafic de la station par rapport à toute la ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
       '00:00-01:00', '01:00-02:00', '02:00-03:00', '03:00-04:00',
       '04:00-05:00', '05:00-06:00', '06:00-07:00', '07:00-08:00',
       '08:00-09:00', '09:00-10:00', '10:00-11:00', '11:00-12:00',
       '12:00-13:00', '13:00-14:00', '14:00-15:00', '15:00-16:00',
       '16:00-17:00', '17:00-18:00', '18:00-19:00', '19:00-20:00',
       '20:00-21:00', '21:00-22:00', '22:00-23:00', '23:00-00:00',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
       'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction',
       'nombre_validations_totales_pour_la_ligne'],
      dtype='object')
Tableau: ligne numéro[1]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,republique,2215941.6,0.007421,24475.263158,4895.052632,2447.526316,5,3,5,8,...,38.42,42.22,48.75,47.65,40.31,26.59,22.13,22.07,20.89,7
1,la defense,9256802.0,0.031002,77993.842105,77993.842105,38996.921053,1,1,nan,nan,...,68.93,79.89,103.14,108.83,84.18,53.23,26.30,14.93,14.40,9
2,hotel de ville,3625864.5,0.012143,15023.947368,7511.973684,3755.986842,2,1,11,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,13
3,palais-royal,2411299.5,0.008076,12183.368421,6091.684211,3045.842105,2,1,7,nan,...,40.91,55.84,70.75,67.08,42.63,27.29,21.24,20.44,20.58,40
4,basilique de saint-denis,3991395.0,0.013367,1889.263158,1889.263158,944.631579,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,jussieu,1444821.0,0.004839,3937.894737,1968.947368,984.473684,2,7,10,nan,...,38.64,47.61,62.35,61.45,41.98,26.16,18.39,17.92,17.77,114
124,mairie d'issy,2874138.0,0.009626,6616.526316,6616.526316,3308.263158,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
125,pernety,2173567.0,0.007279,4404.263158,4404.263158,2202.131579,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
126,notre-dame-de-lorette,1893828.0,0.006343,3949.578947,3949.578947,1974.789474,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188




Tableau: ligne numéro[2]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.009798,9636.526316,3212.175439,1606.087719,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.013077,6307.736842,3153.868421,1576.934211,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,victor hugo,2.801041e+06,0.024207,5284.210526,5284.210526,2642.105263,1,2,nan,nan,...,39.10,43.64,48.86,48.08,43.57,22.99,12.69,12.26,12.33,123
3,alexandre dumas,2.476608e+06,0.021403,4649.315789,4649.315789,2324.657895,1,2,nan,nan,...,36.13,39.34,43.01,39.28,33.30,22.87,14.42,12.51,10.12,137
4,abbesses,1.513884e+06,0.013083,2277.315789,2277.315789,1138.657895,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
5,trinite-d'estienne d'orves,1.051982e+06,0.009091,3454.789474,3454.789474,1727.394737,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
6,saint-georges,7.515550e+05,0.006495,1449.105263,1449.105263,724.552632,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
7,assemblee nationale,6.115120e+05,0.005285,1602.263158,1602.263158,801.131579,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
8,saint-lazare,6.625677e+06,0.057259,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,nation,1.512699e+06,0.013073,19465.157895,4866.289474,2433.144737,4,1,2,6,...,73.30,78.47,90.17,88.67,79.27,49.96,33.56,27.09,22.85,20




Tableau: ligne numéro[3]


...

,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.018686,9636.526316,3212.175439,1606.087719,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.024939,6307.736842,3153.868421,1576.934211,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,abbesses,1.513884e+06,0.024951,2277.315789,2277.315789,1138.657895,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
3,trinite-d'estienne d'orves,1.051982e+06,0.017338,3454.789474,3454.789474,1727.394737,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
4,saint-georges,7.515550e+05,0.012387,1449.105263,1449.105263,724.552632,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
5,assemblee nationale,6.115120e+05,0.010079,1602.263158,1602.263158,801.131579,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
6,saint-lazare,6.625677e+06,0.109201,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
8,marcadet-poissonniers,1.991002e+06,0.032815,9417.421053,4708.710526,2354.355263,2,4,12,nan,...,37.81,39.17,43.07,42.94,37.58,27.00,16.68,10.82,7.56,55
9,convention,3.734750e+06,0.061554,7157.157895,7157.157895,3578.578947,1,12,nan,nan,...,38.31,38.63,37.75,38.46,33.51,20.97,13.06,9.80,7.74,67
10,pigalle,1.750916e+06,0.028858,6591.263158,3295.631579,1647.815789,2,2,12,nan,...,34.97,37.66,39.98,40.12,36.95,26.15,21.85,28.69,26.75,77




Tableau: ligne numéro[13]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,basilique de saint-denis,3991395.0,0.048066,1889.263158,1889.263158,944.631579,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
1,plaisance,3521753.0,0.042411,7378.736842,7378.736842,3689.368421,1,13,nan,nan,...,34.61,37.11,39.72,38.27,32.66,22.15,12.53,8.94,7.01,76
2,malakoff-plateau de vanves,2242320.0,0.027003,4647.684211,4647.684211,2323.842105,1,13,nan,nan,...,35.59,34.75,39.62,41.65,31.21,18.21,9.74,7.87,5.92,155
3,guy moquet,2205313.0,0.026557,5378.578947,5378.578947,2689.289474,1,13,nan,nan,...,34.54,35.67,37.81,36.78,33.77,20.83,12.24,9.86,8.41,158
4,champs-elysees-clemenceau,954502.5,0.011495,4301.315789,2150.657895,1075.328947,2,1,13,nan,...,34.39,47.75,62.33,67.18,52.39,31.77,25.55,24.32,23.10,187
5,les agnettes,1890356.0,0.022765,4123.315789,4123.315789,2061.657895,1,13,nan,nan,...,32.00,33.13,34.57,31.37,25.58,17.10,10.05,7.16,5.00,189
6,la fourche,1829271.0,0.022029,3468.052632,3468.052632,1734.026316,1,13,nan,nan,...,33.81,36.30,40.81,40.02,37.61,24.24,15.98,12.67,11.38,198
7,varenne,782697.0,0.009426,1845.105263,1845.105263,922.552632,1,13,nan,nan,...,39.96,55.50,69.74,68.24,34.46,17.61,11.87,10.16,16.57,290
8,saint-lazare,6625676.8,0.079790,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,chatillon-montrouge,5034012.0,0.060622,11674.526316,11674.526316,5837.263158,1,13,nan,nan,...,32.62,34.96,41.77,41.56,29.17,16.87,9.27,6.52,5.61,35




Tableau: ligne numéro[14]


,Station,Trafic,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nb_correspondance,Correspondance_1,Correspondance_2,Correspondance_3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-lazare,6625676.8,0.119400,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
2,mairie de saint-ouen,2415405.0,0.043528,12017.736842,6008.868421,3004.434211,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39
3,pont cardinet,4168538.0,0.075121,9483.263158,9483.263158,4741.631579,1,14,nan,nan,...,35.56,37.63,43.22,45.32,37.99,23.12,14.08,7.68,6.32,50
4,pyramides,1981349.0,0.035706,8558.263158,4279.131579,2139.565789,2,7,14,nan,...,40.24,48.26,63.20,67.67,51.92,33.49,24.30,22.48,18.01,56
5,clichy saint-ouen,3420852.0,0.061647,9117.842105,9117.842105,4558.921053,1,14,nan,nan,...,34.42,42.10,54.71,54.96,36.11,20.36,12.13,6.13,4.06,84
6,bibliotheque,11104474.0,0.200112,6088.000000,6088.000000,3044.000000,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
7,chatelet,1670158.8,0.030098,87779.526316,17555.905263,8777.952632,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
8,olympiades,5214595.0,0.093971,12997.142857,12997.142857,6498.571429,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
9,gare de lyon,9546825.0,0.172042,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
11,bercy,1942106.0,0.034998,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61


In [17]:
# Définir le nouvel ordre des colonnes
nouvel_ordre_colonnes = [
    'Station',
    'Trafic', 
    'Part du trafic de la station par rapport à toute la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction', 'nb_correspondance',
    'Correspondance_1',
    'Correspondance_2',
    'Correspondance_3',
    'Correspondance_4',
    'Correspondance_5',
    'nombre_validations_totales_pour_la_ligne','fréquentation_totale_ligne',
    '00:00-01:00',
    '01:00-02:00',
    '02:00-03:00',
    '03:00-04:00',
    '04:00-05:00',
    '05:00-06:00',
    '06:00-07:00',
    '07:00-08:00',
    '08:00-09:00',
    '09:00-10:00',
    '10:00-11:00',
    '11:00-12:00',
    '12:00-13:00',
    '13:00-14:00',
    '14:00-15:00',
    '15:00-16:00',
    '16:00-17:00',
    '17:00-18:00',
    '18:00-19:00',
    '19:00-20:00',
    '20:00-21:00',
    '21:00-22:00',
    '22:00-23:00',
    '23:00-00:00',
    'Rang'
]

# Appliquer le nouvel ordre des colonnes à chaque tableau
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau[nouvel_ordre_colonnes]

# Vérifier que rien n'a été oublié
print("Colonnes disponibles :")


Colonnes disponibles :


In [18]:
# Nouveaux noms des colonnes
nouveaux_noms_colonnes = {
    'Station': 'Station',
    'Trafic': 'Trafic annuel dans la station',
    'Part du trafic de la station par rapport à toute la ligne': 'Part de la station dans le trafic annuel de la ligne',
    'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station': 'Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO',
    'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction': 'Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO',
    'Correspondance_1': 'Correspondance 1',
    'Correspondance_2': 'Correspondance 2',
    'Correspondance_3': 'Correspondance 3',
    'Correspondance_4': 'Correspondance 4',
    'Correspondance_5': 'Correspondance 5',
    'nb_correspondance': 'Nombre de correspondances de la station',
    'nombre_validations_totales_pour_la_ligne': 'Nombre de validations annuelles sur la ligne',
    'fréquentation_totale_ligne': 'Fréquentation totale de la ligne',
    '00:00-01:00': '00:00-01:00',
    '01:00-02:00': '01:00-02:00',
    '02:00-03:00': '02:00-03:00',
    '03:00-04:00': '03:00-04:00',
    '04:00-05:00': '04:00-05:00',
    '05:00-06:00': '05:00-06:00',
    '06:00-07:00': '06:00-07:00',
    '07:00-08:00': '07:00-08:00',
    '08:00-09:00': '08:00-09:00',
    '09:00-10:00': '09:00-10:00',
    '10:00-11:00': '10:00-11:00',
    '11:00-12:00': '11:00-12:00',
    '12:00-13:00': '12:00-13:00',
    '13:00-14:00': '13:00-14:00',
    '14:00-15:00': '14:00-15:00',
    '15:00-16:00': '15:00-16:00',
    '16:00-17:00': '16:00-17:00',
    '17:00-18:00': '17:00-18:00',
    '18:00-19:00': '18:00-19:00',
    '19:00-20:00': '19:00-20:00',
    '20:00-21:00': '20:00-21:00',
    '21:00-22:00': '21:00-22:00',
    '22:00-23:00': '22:00-23:00',
    '23:00-00:00': '23:00-00:00',
    'Rang': 'Rang'
}

# Afficher chaque tableau final avec le nouvel ordre des colonnes et les nouveaux noms
for tableau_nom, tableau in tableaux.items():
    print(f"Tableau: {tableau_nom}")
    
    # Réorganiser les colonnes selon le nouvel ordre
    tableau = tableau[nouvel_ordre_colonnes]
    
    # Renommer les colonnes
    tableau = tableau.rename(columns=nouveaux_noms_colonnes)
    
    display(tableau)
    print("\n" + "=" * 50 + "\n")


Tableau: ligne numéro[1]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,republique,2215941.6,0.007421,24475.263158,4895.052632,2447.526316,5,3,5,8,...,38.42,42.22,48.75,47.65,40.31,26.59,22.13,22.07,20.89,7
1,la defense,9256802.0,0.031002,77993.842105,77993.842105,38996.921053,1,1,nan,nan,...,68.93,79.89,103.14,108.83,84.18,53.23,26.30,14.93,14.40,9
2,hotel de ville,3625864.5,0.012143,15023.947368,7511.973684,3755.986842,2,1,11,nan,...,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49,13
3,palais-royal,2411299.5,0.008076,12183.368421,6091.684211,3045.842105,2,1,7,nan,...,40.91,55.84,70.75,67.08,42.63,27.29,21.24,20.44,20.58,40
4,basilique de saint-denis,3991395.0,0.013367,1889.263158,1889.263158,944.631579,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,jussieu,1444821.0,0.004839,3937.894737,1968.947368,984.473684,2,7,10,nan,...,38.64,47.61,62.35,61.45,41.98,26.16,18.39,17.92,17.77,114
124,mairie d'issy,2874138.0,0.009626,6616.526316,6616.526316,3308.263158,1,12,nan,nan,...,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20,115
125,pernety,2173567.0,0.007279,4404.263158,4404.263158,2202.131579,1,13,nan,nan,...,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96,162
126,notre-dame-de-lorette,1893828.0,0.006343,3949.578947,3949.578947,1974.789474,1,12,nan,nan,...,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33,188




Tableau: ligne numéro[2]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.009798,9636.526316,3212.175439,1606.087719,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.013077,6307.736842,3153.868421,1576.934211,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,victor hugo,2.801041e+06,0.024207,5284.210526,5284.210526,2642.105263,1,2,nan,nan,...,39.10,43.64,48.86,48.08,43.57,22.99,12.69,12.26,12.33,123
3,alexandre dumas,2.476608e+06,0.021403,4649.315789,4649.315789,2324.657895,1,2,nan,nan,...,36.13,39.34,43.01,39.28,33.30,22.87,14.42,12.51,10.12,137
4,abbesses,1.513884e+06,0.013083,2277.315789,2277.315789,1138.657895,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
5,trinite-d'estienne d'orves,1.051982e+06,0.009091,3454.789474,3454.789474,1727.394737,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
6,saint-georges,7.515550e+05,0.006495,1449.105263,1449.105263,724.552632,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
7,assemblee nationale,6.115120e+05,0.005285,1602.263158,1602.263158,801.131579,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
8,saint-lazare,6.625677e+06,0.057259,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,nation,1.512699e+06,0.013073,19465.157895,4866.289474,2433.144737,4,1,2,6,...,73.30,78.47,90.17,88.67,79.27,49.96,33.56,27.09,22.85,20




Tableau: ligne numéro[3]


...

,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,concorde,1.133740e+06,0.018686,9636.526316,3212.175439,1606.087719,3,1,8,12,...,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88,86
1,pasteur,1.513143e+06,0.024939,6307.736842,3153.868421,1576.934211,2,6,12,nan,...,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56,105
2,abbesses,1.513884e+06,0.024951,2277.315789,2277.315789,1138.657895,1,12,nan,nan,...,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35,233
3,trinite-d'estienne d'orves,1.051982e+06,0.017338,3454.789474,3454.789474,1727.394737,1,12,nan,nan,...,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85,280
4,saint-georges,7.515550e+05,0.012387,1449.105263,1449.105263,724.552632,1,12,nan,nan,...,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00,291
5,assemblee nationale,6.115120e+05,0.010079,1602.263158,1602.263158,801.131579,1,12,nan,nan,...,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23,296
6,saint-lazare,6.625677e+06,0.109201,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
8,marcadet-poissonniers,1.991002e+06,0.032815,9417.421053,4708.710526,2354.355263,2,4,12,nan,...,37.81,39.17,43.07,42.94,37.58,27.00,16.68,10.82,7.56,55
9,convention,3.734750e+06,0.061554,7157.157895,7157.157895,3578.578947,1,12,nan,nan,...,38.31,38.63,37.75,38.46,33.51,20.97,13.06,9.80,7.74,67
10,pigalle,1.750916e+06,0.028858,6591.263158,3295.631579,1647.815789,2,2,12,nan,...,34.97,37.66,39.98,40.12,36.95,26.15,21.85,28.69,26.75,77




Tableau: ligne numéro[13]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,basilique de saint-denis,3991395.0,0.048066,1889.263158,1889.263158,944.631579,1,13,nan,nan,...,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61,54
1,plaisance,3521753.0,0.042411,7378.736842,7378.736842,3689.368421,1,13,nan,nan,...,34.61,37.11,39.72,38.27,32.66,22.15,12.53,8.94,7.01,76
2,malakoff-plateau de vanves,2242320.0,0.027003,4647.684211,4647.684211,2323.842105,1,13,nan,nan,...,35.59,34.75,39.62,41.65,31.21,18.21,9.74,7.87,5.92,155
3,guy moquet,2205313.0,0.026557,5378.578947,5378.578947,2689.289474,1,13,nan,nan,...,34.54,35.67,37.81,36.78,33.77,20.83,12.24,9.86,8.41,158
4,champs-elysees-clemenceau,954502.5,0.011495,4301.315789,2150.657895,1075.328947,2,1,13,nan,...,34.39,47.75,62.33,67.18,52.39,31.77,25.55,24.32,23.10,187
5,les agnettes,1890356.0,0.022765,4123.315789,4123.315789,2061.657895,1,13,nan,nan,...,32.00,33.13,34.57,31.37,25.58,17.10,10.05,7.16,5.00,189
6,la fourche,1829271.0,0.022029,3468.052632,3468.052632,1734.026316,1,13,nan,nan,...,33.81,36.30,40.81,40.02,37.61,24.24,15.98,12.67,11.38,198
7,varenne,782697.0,0.009426,1845.105263,1845.105263,922.552632,1,13,nan,nan,...,39.96,55.50,69.74,68.24,34.46,17.61,11.87,10.16,16.57,290
8,saint-lazare,6625676.8,0.079790,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
10,chatillon-montrouge,5034012.0,0.060622,11674.526316,11674.526316,5837.263158,1,13,nan,nan,...,32.62,34.96,41.77,41.56,29.17,16.87,9.27,6.52,5.61,35




Tableau: ligne numéro[14]


,Station,Trafic annuel dans la station,Part de la station dans le trafic annuel de la ligne,"Moyenne de la fréquentation hebdomaire de la station en 2022, sur la période des JO (24-07-2022 au 11-08-2022)","Moyenne de la fréquentation hebdomadaire de chaque correspondance de la station en 2022, sur la période des JO","Moyenne de la fréquentation hebdomadaire de chaque correspondance, pour chaque direction en 2022, sur la période des JO",Nombre de correspondances de la station,Correspondance 1,Correspondance 2,Correspondance 3,...,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,Rang
0,saint-lazare,6625676.8,0.119400,127163.368421,25432.673684,12716.336842,5,3,9,12,...,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86,2
2,mairie de saint-ouen,2415405.0,0.043528,12017.736842,6008.868421,3004.434211,2,13,14,nan,...,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66,39
3,pont cardinet,4168538.0,0.075121,9483.263158,9483.263158,4741.631579,1,14,nan,nan,...,35.56,37.63,43.22,45.32,37.99,23.12,14.08,7.68,6.32,50
4,pyramides,1981349.0,0.035706,8558.263158,4279.131579,2139.565789,2,7,14,nan,...,40.24,48.26,63.20,67.67,51.92,33.49,24.30,22.48,18.01,56
5,clichy saint-ouen,3420852.0,0.061647,9117.842105,9117.842105,4558.921053,1,14,nan,nan,...,34.42,42.10,54.71,54.96,36.11,20.36,12.13,6.13,4.06,84
6,bibliotheque,11104474.0,0.200112,6088.000000,6088.000000,3044.000000,1,14,nan,nan,...,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76,6
7,chatelet,1670158.8,0.030098,87779.526316,17555.905263,8777.952632,5,1,4,7,...,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60,10
8,olympiades,5214595.0,0.093971,12997.142857,12997.142857,6498.571429,1,14,nan,nan,...,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47,31
9,gare de lyon,9546825.0,0.172042,66037.947368,22012.649123,11006.324561,3,1,14,,...,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80,3
11,bercy,1942106.0,0.034998,9023.947368,4511.973684,2255.986842,2,6,14,nan,...,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09,61


### PROBLÈMES À RÉGLER : METTRE LES COLONNES DE CHAQUE TABLEAU DANS UN MEILLEUR ORDRE ET RENOMMER LES COLONNES POUR QUE CE SOIT + JOLI ### 

Trafic = le trafic annuel dans la station ; nb_correspondances = le nb de lignes dans la station ; fréquentation totale ligne = trafic annuel / nb_correspondances ; part du trafic de la station par rapport à toute la ligne = trafic / fréquentation_totale_ligne ; les titres de colonnes avec période_jo_2022 : moyenne hebdomadaire sur la période du 24-07-2022 au 11-08-2022

### TABLEAU A EXPLOITER CI DESSUS ###

In [19]:
# Créer le dictionnaire pour les nouvelles variables
tableaux_pour_variables = {}

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes spécifiées
    columns_to_drop = ['Rang', 'Trafic', 'nb_correspondance', 'fréquentation_totale_ligne', 
                       'Part du trafic de la station par rapport à toute la ligne',
                       'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station']

    # Créer un nouveau tableau sans les colonnes spécifiées
    new_table = current_table.drop(columns=columns_to_drop)

    # Ajouter le nouveau tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = new_table

    # Afficher le tableau mis à jour
    print(f"Tableau 'ligne numéro[{k_str}]' sans les colonnes spécifiées :")
    display(tableaux[f'ligne numéro[{k_str}]'])


Tableau 'ligne numéro[1]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,republique,4895.052632,2447.526316,3,5,8,9.0,11.0,756681.396934,13.25,...,32.64,38.42,42.22,48.75,47.65,40.31,26.59,22.13,22.07,20.89
1,la defense,77993.842105,38996.921053,1,nan,nan,nan,nan,756681.396934,7.07,...,62.75,68.93,79.89,103.14,108.83,84.18,53.23,26.30,14.93,14.40
2,hotel de ville,7511.973684,3755.986842,1,11,nan,nan,nan,756681.396934,12.16,...,28.53,39.11,50.70,60.08,62.00,53.19,32.02,19.18,19.01,18.49
3,palais-royal,6091.684211,3045.842105,1,7,nan,nan,nan,756681.396934,9.30,...,31.31,40.91,55.84,70.75,67.08,42.63,27.29,21.24,20.44,20.58
4,basilique de saint-denis,1889.263158,944.631579,13,nan,nan,nan,nan,756681.396934,1.98,...,33.36,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,jussieu,1968.947368,984.473684,7,10,nan,nan,nan,756681.396934,12.91,...,28.96,38.64,47.61,62.35,61.45,41.98,26.16,18.39,17.92,17.77
124,mairie d'issy,6616.526316,3308.263158,12,nan,nan,nan,nan,756681.396934,2.55,...,33.94,34.97,36.23,38.97,38.91,30.74,16.37,8.48,6.66,5.20
125,pernety,4404.263158,2202.131579,13,nan,nan,nan,nan,756681.396934,5.99,...,33.18,33.91,33.31,34.66,34.29,31.53,21.05,13.67,12.71,10.96
126,notre-dame-de-lorette,3949.578947,1974.789474,12,nan,nan,nan,nan,756681.396934,10.62,...,31.34,35.01,35.65,44.18,49.69,39.57,25.65,17.51,18.02,18.33


Tableau 'ligne numéro[2]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,concorde,3212.175439,1606.087719,1,8,12,nan,nan,297028.650658,10.30,...,29.20,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88
1,pasteur,3153.868421,1576.934211,6,12,nan,nan,nan,297028.650658,7.42,...,33.11,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56
2,victor hugo,5284.210526,2642.105263,2,nan,nan,nan,nan,297028.650658,7.13,...,33.19,39.10,43.64,48.86,48.08,43.57,22.99,12.69,12.26,12.33
3,alexandre dumas,4649.315789,2324.657895,2,nan,nan,nan,nan,297028.650658,6.21,...,34.49,36.13,39.34,43.01,39.28,33.30,22.87,14.42,12.51,10.12
4,abbesses,2277.315789,1138.657895,12,nan,nan,nan,nan,297028.650658,10.18,...,34.04,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35
5,trinite-d'estienne d'orves,3454.789474,1727.394737,12,nan,nan,nan,nan,297028.650658,12.18,...,30.97,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85
6,saint-georges,1449.105263,724.552632,12,nan,nan,nan,nan,297028.650658,9.83,...,32.55,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00
7,assemblee nationale,1602.263158,801.131579,12,nan,nan,nan,nan,297028.650658,14.18,...,27.26,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23
8,saint-lazare,25432.673684,12716.336842,3,9,12,13.0,14.0,297028.650658,8.62,...,23.72,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86
10,nation,4866.289474,2433.144737,1,2,6,9.0,nan,297028.650658,15.32,...,66.01,73.30,78.47,90.17,88.67,79.27,49.96,33.56,27.09,22.85


Tableau 'ligne numéro[3]' sans les colonnes spécifiées :


...

,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,concorde,3212.175439,1606.087719,1,8,12,nan,nan,152539.641886,10.30,...,29.20,40.62,54.21,66.58,69.70,54.94,27.15,19.30,18.36,17.88
1,pasteur,3153.868421,1576.934211,6,12,nan,nan,nan,152539.641886,7.42,...,33.11,35.89,38.08,43.50,44.12,35.49,20.42,12.83,11.10,10.56
2,abbesses,2277.315789,1138.657895,12,nan,nan,nan,nan,152539.641886,10.18,...,34.04,40.12,43.49,45.59,43.24,37.00,23.36,19.02,19.36,18.35
3,trinite-d'estienne d'orves,3454.789474,1727.394737,12,nan,nan,nan,nan,152539.641886,12.18,...,30.97,38.18,44.55,51.19,54.76,49.49,33.64,26.85,21.44,16.85
4,saint-georges,1449.105263,724.552632,12,nan,nan,nan,nan,152539.641886,9.83,...,32.55,37.65,38.83,47.79,42.88,34.41,21.11,16.17,26.32,17.00
5,assemblee nationale,1602.263158,801.131579,12,nan,nan,nan,nan,152539.641886,14.18,...,27.26,35.53,47.38,65.30,64.65,41.95,25.79,16.80,15.93,18.23
6,saint-lazare,25432.673684,12716.336842,3,9,12,13.0,14.0,152539.641886,8.62,...,23.72,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86
8,marcadet-poissonniers,4708.710526,2354.355263,4,12,nan,nan,nan,152539.641886,4.49,...,35.76,37.81,39.17,43.07,42.94,37.58,27.00,16.68,10.82,7.56
9,convention,7157.157895,3578.578947,12,nan,nan,nan,nan,152539.641886,4.27,...,36.46,38.31,38.63,37.75,38.46,33.51,20.97,13.06,9.80,7.74
10,pigalle,3295.631579,1647.815789,2,12,nan,nan,nan,152539.641886,19.27,...,28.77,34.97,37.66,39.98,40.12,36.95,26.15,21.85,28.69,26.75


Tableau 'ligne numéro[13]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,basilique de saint-denis,1889.263158,944.631579,13,nan,nan,nan,nan,196412.634211,1.98,...,33.36,34.14,36.42,39.92,33.92,25.82,14.12,8.43,6.61,4.61
1,plaisance,7378.736842,3689.368421,13,nan,nan,nan,nan,196412.634211,4.13,...,33.21,34.61,37.11,39.72,38.27,32.66,22.15,12.53,8.94,7.01
2,malakoff-plateau de vanves,4647.684211,2323.842105,13,nan,nan,nan,nan,196412.634211,2.93,...,34.33,35.59,34.75,39.62,41.65,31.21,18.21,9.74,7.87,5.92
3,guy moquet,5378.578947,2689.289474,13,nan,nan,nan,nan,196412.634211,5.08,...,33.44,34.54,35.67,37.81,36.78,33.77,20.83,12.24,9.86,8.41
4,champs-elysees-clemenceau,2150.657895,1075.328947,1,13,nan,nan,nan,196412.634211,15.47,...,24.47,34.39,47.75,62.33,67.18,52.39,31.77,25.55,24.32,23.10
5,les agnettes,4123.315789,2061.657895,13,nan,nan,nan,nan,196412.634211,2.65,...,31.45,32.00,33.13,34.57,31.37,25.58,17.10,10.05,7.16,5.00
6,la fourche,3468.052632,1734.026316,13,nan,nan,nan,nan,196412.634211,7.98,...,31.75,33.81,36.30,40.81,40.02,37.61,24.24,15.98,12.67,11.38
7,varenne,1845.105263,922.552632,13,nan,nan,nan,nan,196412.634211,6.08,...,32.94,39.96,55.50,69.74,68.24,34.46,17.61,11.87,10.16,16.57
8,saint-lazare,25432.673684,12716.336842,3,9,12,13.0,14.0,196412.634211,8.62,...,23.72,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86
10,chatillon-montrouge,11674.526316,5837.263158,13,nan,nan,nan,nan,196412.634211,2.28,...,32.19,32.62,34.96,41.77,41.56,29.17,16.87,9.27,6.52,5.61


Tableau 'ligne numéro[14]' sans les colonnes spécifiées :


,Station,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,nombre_validations_totales_pour_la_ligne,00:00-01:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00
0,saint-lazare,25432.673684,12716.336842,3,9,12,13.0,14.0,131269.920635,8.62,...,23.72,29.05,39.08,57.36,63.81,51.34,34.47,21.65,16.09,13.86
2,mairie de saint-ouen,6008.868421,3004.434211,13,14,nan,nan,nan,131269.920635,7.75,...,62.26,67.36,77.62,93.80,89.63,63.45,40.80,24.30,16.35,13.66
3,pont cardinet,9483.263158,4741.631579,14,nan,nan,nan,nan,131269.920635,3.67,...,32.76,35.56,37.63,43.22,45.32,37.99,23.12,14.08,7.68,6.32
4,pyramides,4279.131579,2139.565789,7,14,nan,nan,nan,131269.920635,6.84,...,31.18,40.24,48.26,63.20,67.67,51.92,33.49,24.30,22.48,18.01
5,clichy saint-ouen,9117.842105,4558.921053,14,nan,nan,nan,nan,131269.920635,2.59,...,31.23,34.42,42.10,54.71,54.96,36.11,20.36,12.13,6.13,4.06
6,bibliotheque,6088.000000,3044.000000,14,nan,nan,nan,nan,131269.920635,6.46,...,30.38,35.33,43.36,54.02,56.83,44.83,26.99,18.84,11.58,8.76
7,chatelet,17555.905263,8777.952632,1,4,7,11.0,14.0,131269.920635,30.17,...,55.16,72.79,92.19,114.31,122.78,105.60,70.85,51.84,47.85,44.60
8,olympiades,12997.142857,6498.571429,14,nan,nan,nan,nan,131269.920635,2.74,...,35.48,38.44,42.90,47.26,41.98,31.78,20.50,13.19,6.90,4.47
9,gare de lyon,22012.649123,11006.324561,1,14,,nan,nan,131269.920635,2.16,...,27.26,34.76,45.22,57.99,60.21,38.73,27.60,17.40,10.24,4.80
11,bercy,4511.973684,2255.986842,6,14,nan,nan,nan,131269.920635,NaN,...,28.84,2.37,109.81,104.32,67.23,40.18,13.57,0.11,14.23,0.09


In [20]:
for k in numéro:
    k_str = str(k)
    tableaux[f'ligne numéro[{k_str}]'].to_csv(f'ligne_numero_{k_str}.csv', index=False) 

In [40]:
def affluence_hors_jo(horaire, station, ligne):
    # Obtenir le tableau correspondant à la ligne
    line_table = tableaux[f'ligne numéro[{ligne}]']

    # Extraire l'heure de l'horaire entré par l'utilisateur
    hour, minute = map(int, horaire.split(':'))

    # Construire la chaîne de plage horaire correspondante
    plage_horaire = f"{hour:02d}:00-{(hour + 1) % 24:02d}:00"

    # Vérifier si la plage horaire se trouve dans les colonnes du tableau
    if plage_horaire not in line_table.columns:
        raise ValueError(f"Plage horaire non trouvée dans le tableau: {plage_horaire}")

    # Récupérer le nombre à la station et à la plage horaire donnée
    affluence_horaire = line_table[line_table['Station'] == station][plage_horaire].iloc[0]

    # Récupérer l'indice de la plage horaire suivante
    next_time_range_index = line_table.columns.get_loc(plage_horaire) + 1
    next_time_range_index = next_time_range_index % len(line_table.columns)  # Assurer la circularité

    # Récupérer la plage horaire suivante
    plage_horaire_suivante = line_table.columns[next_time_range_index]

    # Récupérer le nombre à la plage horaire suivante
    affluence_suivante = line_table[line_table['Station'] == station][plage_horaire_suivante].iloc[0]

    # Récupérer la moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction à la station
    moyenne_ligne = line_table[line_table['Station'] == station]['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'].iloc[0]

    return round(affluence_horaire), round(affluence_suivante), round(moyenne_ligne)

# Exemple d'utilisation
horaire_info = "01:35"
station_info = "bastille"
ligne_info = "1"

try:
    affluence_horaire, affluence_suivante, affluence_ligne = affluence_hors_jo(horaire_info, station_info, ligne_info)
    print(f"Affluence à {horaire_info} à la station {station_info}, sur la ligne numéro {ligne_info} : {affluence_horaire}")
    print(f"Affluence à la plage horaire suivante à la station {station_info}, sur la ligne numéro {ligne_info} : {affluence_suivante}")
    print(f"Affluence sur la ligne {ligne_info} : {affluence_ligne}")
except ValueError as e:
    print(e)


Affluence à 01:35 à la station bastille, sur la ligne numéro 1 : 8
Affluence à la plage horaire suivante à la station bastille, sur la ligne numéro 1 : 0
Affluence sur la ligne 1 : 2633
